# Interfaces

In python (since 3.8) we have four types of seeing a inteface:
- Duck typing: as we have seen before
- Goose typing: supported by the abstract (abc) classes, which depends on verifying against interfaces at runtime
- Static typing: traditional way that languages like Java use. Supported by the `typing` module and verified by `mypy`
- Static duck typing: using the `typing.Protocol`

![Typing map](image.png)

# Two types of protocol

- Dynamic: informal interface that is described in the documentation. 
- Static: Defined using `typing.Protocol`.

Both types of protocol shares that they don't need to be used with inheritance.

# Defensive programming 

A lot of bugs cannot be found unless we are at execution time. Because of this, it's better to fail faster. For example, discarding not valid arguments in the beginning of a function.

Using the `list()` constructor at the `__init__()` method it's another example to be used when it's ok to copy the input.

When we need to check if a input is able to be modified internally, we can use `isinstance(x, abc.MutableSequence)`

Call `len()` to discard iterators

Call `iter()` to confirm that a variable is an interator.

Emulating the creation of `namedtuple`:

```python
def namedtupel(typename: str, field_names: Union[str, Iterable[str]], *):
try: # assumes that the input is a str
    field_names = field_names.replace(',', ' ').split()
except AttributeError: # It was not a str, assumes it's a iterable
    pass
field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names)
    raise ValueError('invalid identifier for field_name')
```

# Goose typing

Python does not have a `interface` keyword. We use the `abc` abstract classes to define them, that can be used by runtime checks like `isinstance` or `issubclass`, or can be used by static checkers like mypy.

Using `isintance(obj, cls)` is acceptable if `cls` is a `abc` because we are comparing against the interface and not the actual class implementation. This is done to avoid losing the polymorphism when comparing with concrete classes. What defines if a object is instance of a class will be the methods that it implements

```python
class Answer:
    def __len__(self): return 42

from collections import abc
isinstance(Answer(), abc.Sized) # True, because of structural typing (see section 13.5.8)
```

Another option is to `register` your concrete class as a subclass of the `abc` module

```python
from collections.abc import Sequence
Sequence.register(FrenchDeck)
```


In [6]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position): # Needed to be implemented because of abc.MutableSequence
        del self._cards[position] 

    def insert(self, position, value): # Same, but why not a dunder item?
        self._cards.insert(position, value) 

deck = FrenchDeck2()
deck.append(Card('42', 'fish'))

Card(rank='A', suit='hearts')

The code above is example of the trade-offs of using the `abc` interface, because we need to implement function that are not necessary in this example, but we have access to functions that are implemented in the super classes, like `append`, `reverse`, etc.

In [10]:
# Defining an interface using abc

# The interface is going to represent a object that can be used
# to pop a random item from a list. It can be used later to implement
# a bingo machine or how to pick a new ad for an app.

import abc
class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable"""

    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it
        
        This should raise 'LookupError' when the instance is empty
        """

    def loaded(self):
        """Return True if there's at least 1 item, False otherwise"""
        return bool(self.inspect())

    def inspect(self):
        """Return a sorted tuple with the items currently inside"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

class Incorrect(Tombola):
    def pick(self):
        return 13

# wrong = Incorrect() # Error because Incorrect doesn't implement load()

import random
class BingoCage(Tombola):
    def __init__(self, items) -> None:
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        self.pick()

    # We could implement inspect and loaded as well.
        
right = BingoCage([1, 2])

from random import randrange

@Tombola.register # same as Tombola.register(TomboList) after the class definition
class TomboList(list):
    def pick(self):
        if self: # list is not empty
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend # function attribution

    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(self)

issubclass(TomboList, Tombola) # True
t = TomboList(100)
isinstance(t, Tombola) # True, however, Tombolist does not inherit any method from Tombola



# Static duck typing

Suppose we want to use type hints for the function bellow:

```python
def double(x):
    return x * 2

double(1.5) # 3.0
double('A') # 'AA'
double([10, 20, 30]) # [10, 20, 30, 10, 20, 30]
```

With `typing.Protocol` we  can now tell `mypy` that `double` function receives a argument that can be repeated.

```python
from typing import TypeVar, Protocol

T = TypeVar('T')

class Repeatable(Protocol):
    def __mul__(self: T, repeat_count: int) -> T: ... # Question: repeat_count is part of __mul__ definition?

RT = TypeVar('RT', bound=Repeatable)

def double(x: RT) -> RT:
    return x * 2
```

## Runtime checks using static typing

Use `@runtime_checkable` to allow a protocol to be checked at runtime with `isinstance/issubclass`.

Examples of Protocols that are available out of the box in python and that are runtime checkable:
- `typing.SupportsComplex`: An ABC with one abstract method `__complex__`
- `typing.SupportsFloat`: An ABC with one abstract method `__float__`

```python
from typing import SupportsComplex
import numpy as np

c64 = np.complex64(3+4j)
isinstance(c64, complex) # False, because 'complex' is builtin type
isinstance(c64, SupportsComplex) # True, because np.complex64 implements __complex__
c = complex(c64) # Available because we have the impl of __complex__
isinstance(c, SupportsComplex) # False, because the builtin type doesn't implement __complex__
isinstance(c, (complex, SupportsComplex)) # To be able to use both np.complex64 and builtin complex
```



In [ ]:
# Using SupportsComplex in the Vector2d class

from typing import SupportsComplex
class Vector2d:
    # A lot of definition

    def __complex__(self) -> complex:
        return complex(self.x, self.y)
    
    @classmethod
    def fromcomplex(cls, datum: SupportsComplex):
        c = complex(datum)
        return cls(c.real, c.imag)

In [ ]:
# Creating a Protocol for replacing Tombola abc interface
# The load function was not converted to a Protocol for simplicity.

from typing import Protocol, runtime_checkable, Any, Iterable, TYPE_CHECKING

@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any: ...

class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:
        return self._items.pop()
    
popper = SimplePicker([1])
isinstance(popper, RandomPicker) # True, at runtime because of @runtime_checkable and because pick() is implemented0

if TYPE_CHECKING: # Only execute when running mypy
    reveal_type(popper.pick()) # Magic function that tells the type of the argument

## Best practices

Use protocols that implements one or a few functions. The simpler the better.

Define the protocol closer to the function that will use it.

For nomenclature:
- Use simple names for protocols with a simple concept. Example: Iterator, Container
- use `SupportsX` for protocols that offers methods that can be called. Example: `SupportInt`
- use `HasX` for protocols that have attributes that can be read or written, or for getter/setter methods.



## Extending a protocol

It's better to extend a protocol (using inheritance) than having a protocol with lots of functions


```python
@runtime_checkable
class LoadableRandomPicker(RandomPicker, Protocol):
    def load(self, Iterable) -> None: ...
```